In [2]:
#this is the first line for committ 

import os
import random
import numpy as np
import pandas as pd
import collections
import math


In [3]:
uri_train  = 'https://raw.githubusercontent.com/thiagorainmaker77/liar_dataset/master/train.tsv'

df_train = pd.read_table(uri_train,
                             names = ['id',	'label'	,'statement',	'subject',	'speaker', 	'job', 	'state',	'party',	'barely_true_c',	'false_c',	'half_true_c',	'mostly_true_c',	'pants_on_fire_c',	'venue'])

other_df = pd.DataFrame({"label": ['true', "false"]})

df_linear = df_train[df_train.label.isin(other_df.label)]

df_linear_1= df_linear.reset_index()
df_linear_1['statement'] = df_linear_1['statement'].str.lower()
#df_linear_1['statement']= df_linear_1['statement'].str.split()


In [4]:
uri_test  = 'https://raw.githubusercontent.com/thiagorainmaker77/liar_dataset/master/test.tsv'

df_test = pd.read_table(uri_train,
                             names = ['id',	'label'	,'statement',	'subject',	'speaker', 	'job', 	'state',	'party',	'barely_true_c',	'false_c',	'half_true_c',	'mostly_true_c',	'pants_on_fire_c',	'venue'])

other_df_1 = pd.DataFrame({"label": ['true', "false"]})

df_linear_test = df_test[df_test.label.isin(other_df_1.label)]

df_linear_test_1= df_linear_test.reset_index()
df_linear_test_1['statement'] = df_linear_test_1['statement'].str.lower()
#df_linear_1['statement']= df_linear_1['statement'].str.split()


In [5]:
label_cnt = collections.Counter(df_linear_1.label) #num of occurances 
print(label_cnt)

Counter({'false': 1995, 'true': 1676})


In [6]:
label_cnt = collections.Counter(df_linear_1.label)
true_cnt = collections.Counter() #true_cnt is p(x|true)
false_cnt = collections.Counter()



for idx, row in df_linear_1.iterrows():

    if row["label"]== "true":

        true_cnt.update(row["statement"].split())

    if row["label"]== "false":

        false_cnt.update(row["statement"].split())




In [7]:
true_cnt

Counter({'the': 1693,
         'chicago': 1,
         'bears': 1,
         'have': 246,
         'had': 68,
         'more': 203,
         'starting': 2,
         'quarterbacks': 2,
         'in': 915,
         'last': 63,
         '10': 30,
         'years': 54,
         'than': 258,
         'total': 12,
         'number': 31,
         'of': 896,
         'tenured': 1,
         '(uw)': 1,
         'faculty': 3,
         'fired': 3,
         'during': 26,
         'two': 36,
         'decades.': 3,
         'mccain': 32,
         'opposed': 11,
         'a': 567,
         'requirement': 1,
         'that': 272,
         'government': 44,
         'buy': 11,
         'american-made': 1,
         'motorcycles.': 1,
         'and': 433,
         'he': 112,
         'said': 49,
         'all': 64,
         'buy-american': 1,
         'provisions': 4,
         'were': 73,
         'quote': 1,
         "'disgraceful.'": 1,
         'says': 319,
         'paul': 8,
         'ryan': 4,
      

In [8]:
true_probs = {key: value / sum(true_cnt.values()) for key, value in true_cnt.items()}
false_probs = {key: value / sum(false_cnt.values()) for key, value in false_cnt.items()}
label_probs = {key: value / sum(label_cnt.values()) for key, value in label_cnt.items()}


In [9]:
#this is a for loop 
#each key is the word and the value is the num of occurance 

tru_probs_1 ={}

for key, value in true_cnt.items():
    #this is all the values again 
    tru_probs_1[key]= value/sum(true_cnt.values()) 


print(tru_probs_1==true_probs)


True


In [10]:
for idx, row in df_linear_1.iterrows(): 

    sentence = row["statement"].split() 

    true_prob = math.prod([true_probs.get(word,0) for word in sentence])*label_probs["true"]
    false_prob = math.prod([false_probs.get(word,0) for word in sentence])*label_probs["false"]

    if true_prob > false_prob: 
        print("true")
    else: 
        print("false")

    print(row["label"])


    break



false
false


In [11]:
#true_probs.get(word,0) for word in sentence

for idx, row in df_linear_1.iterrows(): 

    sentence = row["statement"].split() 

    #true_prob = math.prod([true_probs.get(word,0) for word in sentence])*label_probs["true"]
    #false_prob = math.prod([false_probs.get(word,0) for word in sentence])*label_probs["false"]

    true_prob=[]
    false_prob=[]

    for word in sentence:
        # we're building an array that contains one value per word in the sentence. 
        # And then we're looking up the probabilities of each of those words and putting them in the array. So for each 
        # word in the sentence, we're looking at the probability of seeing that word in any sentence, and the 
        # true sentence, in this case. 
        true_prob.append(true_probs.get(word,0))

        # So dot get is the same as doing square brackets, except how it handles the situation that the word 
        # doesn't exist. So when the word doesn't exist, if you use square brackets, it'll throw an exception 
        # it'll say key error.  Can't find that word. If you use that get instead of throwing an exception, 
        # It will allow you to specify a default value to return If the word doesn't exist in the true probabilities 
        # collection. So this is just specifying that the default if that word doesn't exist in the collection is zero. 
        # the probability of this word existing in this work. The probability of this word being in the true probabilities 
        # array is zero, if it doesn't exist in that array

    true_prob= math.prod(true_prob)*label_probs["true"]

    for word in sentence: 
        false_prob.append(false_probs.get(word,0))

    false_prob= math.prod(false_prob)*label_probs["false"]



    if true_prob > false_prob: 
        print("true")
    else: 
        print("false")

    print(row["label"])


    break


false
false


runnig things on test set 

In [12]:
#we need to generate an empty to deposit results as we move through 

to_calc =[]

for idx, row in df_linear_test_1.iterrows(): 

    sentence = row["statement"].split() 
    #p(X|y)p(y) = p(x1|y)*…*p(xn|y)p(y)
    true_prob = math.prod([true_probs.get(word,0) for word in sentence])*label_probs["true"]
    false_prob = math.prod([false_probs.get(word,0) for word in sentence])*label_probs["false"]

    if true_prob > false_prob: 
        curr_probs = "true"
    else: 
        curr_probs = "false"

    #this is to calc accurracy later 
    if curr_probs == row["label"]: 
        to_calc.append(1)
    else: 
        to_calc.append(0)




    #print(row["label"])



print(to_calc)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [13]:
count_correct = to_calc.count(1)
rows_count = df_linear_test_1.shape[0]
accuracy = count_correct/rows_count
accuracy

0.9670389539634977

This is the part where we do the sum of log probabilities 

In [17]:
true_prob_log = dict((k, math.log(v)) for k,v in true_probs.items())
false_prob_log = dict((k, math.log(v)) for k,v in false_probs.items())

In [26]:

to_calc_1 =[]

for idx, row in df_linear_test_1.iterrows(): 

    sentence = row["statement"].split() 

    true_prob = sum([true_prob_log.get(word,float("-inf")) for word in sentence])+math.log(label_probs["true"])
    false_prob = sum([false_prob_log.get(word,float("-inf")) for word in sentence])+math.log(label_probs["false"])

    #float("-inf")- this is to deal with log(0) situation 

    if true_prob > false_prob: 
        curr_probs = "true"
    else: 
        curr_probs = "false"

    #this is to calc accurracy later 
    if curr_probs == row["label"]: 
        to_calc_1.append(1)
    else: 
        to_calc_1.append(0)




    #print(row["label"])



print(to_calc_1)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [27]:
count_correct = to_calc_1.count(1)
rows_count = df_linear_test_1.shape[0]
accuracy = count_correct/rows_count
accuracy

0.9673113593026423